## 1. Abstract Base Classes (ABC)

In [ ]:
from abc import ABC, abstractmethod

class Animal(ABC):
    """Abstract base class - cannot be instantiated directly"""
    
    def __init__(self, name):
        self.name = name
    
    @abstractmethod
    def speak(self):
        """All animals must implement this"""
        pass
    
    @abstractmethod
    def move(self):
        """All animals must implement this"""
        pass
    
    # Concrete method - shared by all
    def describe(self):
        return f"{self.name} the {self.__class__.__name__}"

# Cannot instantiate abstract class
try:
    animal = Animal("Generic")
except TypeError as e:
    print(f"❌ Cannot create Animal: {e}")

# Concrete implementations
class Dog(Animal):
    def speak(self):
        return "Woof!"
    
    def move(self):
        return "runs on four legs"

class Bird(Animal):
    def speak(self):
        return "Tweet!"
    
    def move(self):
        return "flies through the air"

# These work
dog = Dog("Buddy")
bird = Bird("Tweety")

print(f"\n{dog.describe()} says {dog.speak()} and {dog.move()}")
print(f"{bird.describe()} says {bird.speak()} and {bird.move()}")

In [ ]:
# Incomplete implementation fails

class Fish(Animal):
    def speak(self):
        return "Blub!"
    # Missing move() method!

try:
    fish = Fish("Nemo")
except TypeError as e:
    print(f"❌ Cannot create Fish: {e}")

# Fix it
class Fish(Animal):
    def speak(self):
        return "Blub!"
    
    def move(self):
        return "swims in water"

fish = Fish("Nemo")
print(f"✅ {fish.describe()} says {fish.speak()} and {fish.move()}")

## 2. Abstract Properties

In [ ]:
from abc import ABC, abstractmethod

class Shape(ABC):
    """Abstract shape with required properties"""
    
    @property
    @abstractmethod
    def area(self):
        """Calculate and return area"""
        pass
    
    @property
    @abstractmethod
    def perimeter(self):
        """Calculate and return perimeter"""
        pass
    
    def describe(self):
        return f"{self.__class__.__name__}: Area={self.area:.2f}, Perimeter={self.perimeter:.2f}"

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    @property
    def area(self):
        return self.width * self.height
    
    @property
    def perimeter(self):
        return 2 * (self.width + self.height)

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def area(self):
        import math
        return math.pi * self.radius ** 2
    
    @property
    def perimeter(self):
        import math
        return 2 * math.pi * self.radius

# Test
shapes = [Rectangle(10, 5), Circle(7)]
for shape in shapes:
    print(shape.describe())

## 3. Interface Pattern

In [ ]:
# Define interfaces (contracts)

class Serializable(ABC):
    """Interface for objects that can be serialized"""
    
    @abstractmethod
    def to_dict(self):
        pass
    
    @classmethod
    @abstractmethod
    def from_dict(cls, data):
        pass

class Printable(ABC):
    """Interface for objects that can be printed"""
    
    @abstractmethod
    def print_format(self):
        pass

# Implement multiple interfaces
class User(Serializable, Printable):
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    # Serializable interface
    def to_dict(self):
        return {'name': self.name, 'email': self.email}
    
    @classmethod
    def from_dict(cls, data):
        return cls(data['name'], data['email'])
    
    # Printable interface
    def print_format(self):
        return f"User: {self.name} <{self.email}>"

# Test
user = User("Alice", "alice@example.com")

# Serialize
data = user.to_dict()
print(f"Serialized: {data}")

# Deserialize
user2 = User.from_dict(data)
print(f"Restored: {user2.print_format()}")

## 4. Protocols (Python 3.8+)

In [ ]:
# Protocols - structural subtyping (duck typing with type hints)
from typing import Protocol, runtime_checkable

@runtime_checkable
class Drawable(Protocol):
    """Protocol - just needs to have draw() method"""
    def draw(self) -> str:
        ...

# No inheritance needed!
class Square:
    def __init__(self, size):
        self.size = size
    
    def draw(self) -> str:
        return f"Drawing square of size {self.size}"

class Text:
    def __init__(self, content):
        self.content = content
    
    def draw(self) -> str:
        return f"Drawing text: {self.content}"

class Button:
    def __init__(self, label):
        self.label = label
    
    def draw(self) -> str:
        return f"Drawing button: [{self.label}]"

# Function accepts anything Drawable
def render(item: Drawable) -> None:
    print(item.draw())

# Works without any inheritance!
items = [Square(10), Text("Hello"), Button("Click Me")]

for item in items:
    print(f"Is Drawable? {isinstance(item, Drawable)}")
    render(item)

## 5. Complete Example: Plugin System

In [ ]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any

# Abstract base for plugins
class Plugin(ABC):
    """Base class for all plugins"""
    
    @property
    @abstractmethod
    def name(self) -> str:
        """Plugin name"""
        pass
    
    @property
    @abstractmethod
    def version(self) -> str:
        """Plugin version"""
        pass
    
    @abstractmethod
    def initialize(self) -> bool:
        """Initialize the plugin"""
        pass
    
    @abstractmethod
    def execute(self, data: Any) -> Any:
        """Execute plugin logic"""
        pass
    
    def cleanup(self) -> None:
        """Optional cleanup (default implementation)"""
        pass

# Concrete plugins
class LoggerPlugin(Plugin):
    @property
    def name(self):
        return "Logger"
    
    @property
    def version(self):
        return "1.0.0"
    
    def initialize(self):
        print(f"  [{self.name}] Initialized logging")
        return True
    
    def execute(self, data):
        print(f"  [{self.name}] LOG: {data}")
        return data

class ValidationPlugin(Plugin):
    @property
    def name(self):
        return "Validator"
    
    @property
    def version(self):
        return "2.1.0"
    
    def initialize(self):
        print(f"  [{self.name}] Loaded validation rules")
        return True
    
    def execute(self, data):
        if isinstance(data, dict) and 'email' in data:
            is_valid = '@' in data.get('email', '')
            data['valid'] = is_valid
            print(f"  [{self.name}] Validated email: {is_valid}")
        return data

class TransformPlugin(Plugin):
    @property
    def name(self):
        return "Transformer"
    
    @property
    def version(self):
        return "1.5.0"
    
    def initialize(self):
        print(f"  [{self.name}] Ready to transform")
        return True
    
    def execute(self, data):
        if isinstance(data, dict):
            # Transform all string values to uppercase
            for key, value in data.items():
                if isinstance(value, str):
                    data[key] = value.upper()
            print(f"  [{self.name}] Transformed data")
        return data

# Plugin manager
class PluginManager:
    def __init__(self):
        self.plugins: List[Plugin] = []
    
    def register(self, plugin: Plugin):
        self.plugins.append(plugin)
        print(f"Registered: {plugin.name} v{plugin.version}")
    
    def initialize_all(self):
        print("\n🔧 Initializing plugins...")
        for plugin in self.plugins:
            if not plugin.initialize():
                print(f"  ❌ {plugin.name} failed to initialize")
    
    def process(self, data: Any) -> Any:
        print("\n⚙️ Processing data through plugins...")
        for plugin in self.plugins:
            data = plugin.execute(data)
        return data
    
    def cleanup_all(self):
        for plugin in self.plugins:
            plugin.cleanup()

# Demo
print("🔌 PLUGIN SYSTEM DEMO")
print("=" * 50)

# Create and register plugins
manager = PluginManager()
manager.register(LoggerPlugin())
manager.register(ValidationPlugin())
manager.register(TransformPlugin())

# Initialize
manager.initialize_all()

# Process data
user_data = {
    'name': 'alice',
    'email': 'alice@example.com',
    'role': 'admin'
}

print(f"\n📥 Input: {user_data}")
result = manager.process(user_data)
print(f"\n📤 Output: {result}")

## Summary

### Abstract Classes vs Protocols:

| Feature | ABC | Protocol |
|---------|-----|----------|
| Inheritance | Required | Not required |
| Type checking | Runtime | Static (type hints) |
| Default methods | Yes | No |
| Python version | Any | 3.8+ |

### Key Decorators:

| Decorator | Purpose |
|-----------|----------|
| `@abstractmethod` | Method must be implemented |
| `@property` + `@abstractmethod` | Property must be implemented |
| `@classmethod` + `@abstractmethod` | Class method must be implemented |

### When to Use:

| Scenario | Use |
|----------|-----|
| Enforce method implementation | ABC |
| Provide shared default behavior | ABC |
| Duck typing with type hints | Protocol |
| Third-party class compatibility | Protocol |

### Best Practices:
1. Use ABCs to define contracts
2. Keep abstract classes focused
3. Prefer composition over inheritance
4. Use Protocols for flexibility
5. Document expected behavior

### Next Lesson: Exception Handling